In [ ]:
import requests
import psycopg2
import psycopg2.extras  # Import the extras module
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta


month_ago = int(time.mktime((datetime.now() - timedelta(days=29)).timetuple()))

ACCESS_TOKEN = 'ppp'

# Base URL for the Facebook Graph API
BASE_URL = "https://graph.facebook.com/v20.0/"

#id is account_id and also page_id DATAFRAME NAME = fb_accounts
f_accounts = "me/accounts?fields=id,access_token,username,name,profile_picture_url,email,biography,description,followers_count,follows_count,website,media_count&access_token={}".format(ACCESS_TOKEN)


def fetch_api_data(url):
    all_data = []
    try:
        while url:
            response = requests.get(url)
            response.raise_for_status()  # Will raise an HTTPError for bad responses
            result = response.json()
            
            if 'data' in result:
                all_data.extend(result['data'])

            if 'paging' in result and 'next' in result['paging']:
                url = result['paging']['next']
            else:
                url = None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
    return all_data

# Fetch account data
accounts_url = f"{BASE_URL}{f_accounts}"

# Fetch the accounts data
fb_accounts_data = fetch_api_data(accounts_url)

# Convert the fetched data to a DataFrame
fb_accounts = pd.DataFrame(fb_accounts_data)


# Loop through accounts and fetch page insights for each

fb_page_insights_data = []

for index, row in fb_accounts.iterrows():
    page_id = row['id']  # Use the page_id from fb_accounts
    page_token = row['access_token']  # Use the page access token
    
    # Define f_page_insights INSIDE the loop with the correct page_id and page_token
    f_page_insights = "{page_id}/insights?metric=page_impressions,page_impressions_unique,page_fan_adds,page_fan_removes,page_fans_online,page_fans_country,page_fans_city,page_views_total,page_post_engagements,page_total_actions,page_consumptions_by_consumption_type,page_negative_feedback,page_negative_feedback_unique,page_posts_impressions,page_posts_impressions_unique,page_posts_impressions_paid,page_posts_impressions_organic,page_post_engagements&access_token={PAGE_TOKEN}&period=day&since={month_ago}".format(page_id=page_id, PAGE_TOKEN=page_token, month_ago=month_ago)
    
    # Build the URL for fetching page insights
    page_insights_url = f"{BASE_URL}{f_page_insights}"

    # Fetch the data
    insights_data = fetch_api_data(page_insights_url)

    # Dictionary to hold combined metrics for the current page and date
    insights_combined = {}

    # Loop through each metric in the insights_data
    for insight in insights_data:
        metric_name = insight['name']
        for value in insight['values']:
            end_time = value['end_time']
            
            # If this end_time isn't already in the combined data, initialize it
            if end_time not in insights_combined:
                insights_combined[end_time] = {
                    'page_id': page_id,
                    'end_date': end_time
                }

            # Add the metric to the corresponding end_time dictionary
            insights_combined[end_time][metric_name] = value['value']

    # Convert combined metrics into rows for the DataFrame
    for end_time, combined_row in insights_combined.items():
        fb_page_insights_data.append(combined_row)

# Convert the insights data to a DataFrame
fb_page_insights = pd.DataFrame(fb_page_insights_data)



# Loop through accounts and fetch page posts for each
fb_posts_data = []

for index, row in fb_accounts.iterrows():
    page_id = row['id']  # Use the page_id from fb_accounts
    page_token = row['access_token']  # Use the page access token
    
    # Define f_posts INSIDE the loop with the correct page_id and page_token
    f_posts = "{page_id}/posts?fields=id,parent_id,promotion_status,scheduled_publish_time,message,message_tags,created_time,permalink_url,status_type,call_to_action,instagram_eligibility,is_published,is_hidden,ad_status,ads_run&access_token={PAGE_TOKEN}".format(page_id=page_id, PAGE_TOKEN=page_token)
    
    # Build the URL for fetching posts
    posts_url = f"{BASE_URL}{f_posts}"

    # Fetch the data
    posts_data = fetch_api_data(posts_url)

    # Loop through the posts in the 'posts' list
    for post in posts_data:
        fb_posts_data.append({
            'page_id': page_id,  # Save the page_id (account_id)
            'page_token': page_token,  # Save the page token
            'post_id': post['id'],
            'parent_id': post.get('parent_id', None),
            'promotion_status': post.get('promotion_status', None),
            'scheduled_publish_time': post.get('scheduled_publish_time', None),
            'message': post.get('message', None),
            'message_tags': json.dumps(post.get('message_tags', None)),  # Serialize JSON tags if available
            'created_time': post.get('created_time', None),
            'permalink_url': post.get('permalink_url', None),
            'status_type': post.get('status_type', None),
            'call_to_action': json.dumps(post.get('call_to_action', None)),  # Serialize JSON call_to_action if available
            'instagram_eligibility': post.get('instagram_eligibility', None),
            'is_published': post.get('is_published', None),
            'is_hidden': post.get('is_hidden', None),
            'ad_status': post.get('ad_status', None),
            'ads_run': post.get('ads_run', None)
        })

# Convert the flattened posts data to a DataFrame
fb_posts = pd.DataFrame(fb_posts_data)


# Loop through posts and fetch post insights for each
fb_post_insight_data = []

for index, row in fb_posts.iterrows():
    post_id = row['post_id']  # Extract the post_id from the current row
    page_id = row['page_id']  # Extract the page_id from the current row
    
    # Retrieve the page_token from the current row (if not already available)
    page_token = row['page_token']
    
    # Define f_post_insight INSIDE the loop with the correct post_id and page_token
    f_post_insight = "{post_id}/insights?metric=post_impressions,post_clicks,post_impressions_paid,post_activity_by_action_type,post_clicks_by_type,page_consumptions_by_consumption_type&access_token={PAGE_TOKEN}&period=day&since={month_ago}".format(post_id=post_id, PAGE_TOKEN=page_token, month_ago=month_ago)
    
    # Build the URL for fetching post insights
    post_insight_url = f"{BASE_URL}{f_post_insight}"

    # Fetch the data
    post_insight_data = fetch_api_data(post_insight_url)

    # Dictionary to hold combined metrics for the current post and date
    post_insights_combined = {}

    # Loop through each metric in the post_insight_data
    for insight in post_insight_data:
        metric_name = insight['name']
        for value in insight['values']:
            end_time = value['end_time']
            
            # If this end_time isn't already in the combined data, initialize it
            if end_time not in post_insights_combined:
                post_insights_combined[end_time] = {
                    'post_id': post_id,
                    'page_id': page_id,
                    'end_date': end_time
                }

            # Add the metric to the corresponding end_time dictionary
            post_insights_combined[end_time][metric_name] = value['value']

    # Convert combined metrics into rows for the DataFrame
    for end_time, combined_row in post_insights_combined.items():
        fb_post_insight_data.append(combined_row)

# Convert the insights data to a DataFrame
fb_post_insights = pd.DataFrame(fb_post_insight_data)


# Loop through posts and fetch post comments for each
fb_post_comments_data = []

for index, row in fb_posts.iterrows():
    post_id = row['post_id']  # Extract the post_id from the current row
    page_id = row['page_id']  # Extract the page_id from the current row
    
    # Retrieve the page_token from the current row (if not already available)
    page_token = row['page_token']
        
    # Define f_post_comments INSIDE the loop with the correct post_id and page_token
    f_post_comments = "{post_id}/comments?fields=from,id,message,created_time,like_count,comment_count,attachment,parent,hidden&access_token={PAGE_TOKEN}".format(post_id=post_id, PAGE_TOKEN=page_token)
        
    # Build the URL for fetching post comments
    post_comments_url = f"{BASE_URL}{f_post_comments}"

    # Fetch the data
    post_comments_data = fetch_api_data(post_comments_url)

    # Loop through each comment in the fetched data
    for comment in post_comments_data:
        from_user = comment.get('from', {}).get('name')  # Extract 'from' field's name
        comment_id = comment['id']  # Comment ID
        message = comment.get('message')  # Message content
        created_time = comment.get('created_time')  # Creation time of the comment
        like_count = comment.get('like_count', 0)  # Number of likes on the comment
        comment_count = comment.get('comment_count', 0)  # Number of replies to the comment
        attachment = json.dumps(comment.get('attachment', {}))  # Serialize attachment to JSON
        parent = comment.get('parent', {}).get('id')  # Parent comment if this is a reply
        hidden = comment.get('hidden', False)  # Whether the comment is hidden

        # Append flattened data to the fb_post_comments_data list
        fb_post_comments_data.append({
            'post_id': post_id,
            'page_token': page_token,  # Save the page token            
            'comment_id': comment_id,
            'from_user': from_user,
            'message': message,
            'created_time': created_time,
            'like_count': like_count,
            'comment_count': comment_count,
            'attachment': attachment,
            'parent': parent,
            'hidden': hidden
        })

# Convert the flattened post comments data to a DataFrame
fb_post_comments = pd.DataFrame(fb_post_comments_data)


fb_comments_data = []

for index, row in fb_post_comments.iterrows():
    comment_id = comment['id']  # Extract the comment_id from the comment data
    post_id = row['post_id']  # Extract the post_id from the current row

    # Retrieve the page_token from the current row
    page_token = row['page_token']

    # Define f_comments INSIDE the loop with the correct comment_id and page_token
    f_comments = "{comment_id}/comments?fields=from,id,message,created_time,like_count,attachment,parent,hidden&access_token={PAGE_TOKEN}".format(comment_id=comment_id, PAGE_TOKEN=page_token)

    # Build the URL for fetching comment replies
    comments_url = f"{BASE_URL}{f_comments}"

    # Fetch the data
    comments_data = fetch_api_data(comments_url)

    # Loop through replies to the comment
    for reply in comments_data:
        reply_id = reply['id']  # Extract the reply_id
        from_user = reply.get('from', {}).get('name')  # Extract the name of the user
        message = reply.get('message')  # Extract the message content
        created_time = reply.get('created_time')  # Extract the creation time
        like_count = reply.get('like_count', 0)  # Number of likes on the reply
        attachment = json.dumps(reply.get('attachment', {}))  # Serialize the attachment field
        parent_comment = reply.get('parent', {}).get('id')  # Parent comment ID, if this is a reply
        hidden = reply.get('hidden', False)  # Whether the reply is hidden
            
        # Append flattened reply data to the fb_comments_data list
        fb_comments_data.append({
            'post_id': post_id,
            'comment_id': comment_id,
            'reply_id': reply_id,
            'from_user': from_user,
            'message': message,
            'created_time': created_time,
            'like_count': like_count,
            'attachment': attachment,
            'parent_comment': parent_comment,
            'hidden': hidden
        })

# Convert the comments data (with replies) to a DataFrame
fb_comments = pd.DataFrame(fb_comments_data)


# Loop through comments and fetch reactions for each
fb_reactions_data = []

for index, row in fb_post_comments.iterrows():
    comment_id = comment['id']  # Extract the comment_id from the comment data
    post_id = row['post_id']  # Extract the post_id from the current row

    # Retrieve the page_token from the current row
    page_token = row['page_token']

    # Define f_reactions INSIDE the loop with the correct comment_id and page_token
    f_reactions = "{comment_id}/reactions?fields=id,name,type&access_token={PAGE_TOKEN}".format(comment_id=comment_id, PAGE_TOKEN=page_token)

    # Build the URL for fetching reactions
    reactions_url = f"{BASE_URL}{f_reactions}"

    # Fetch the data
    reactions_data = fetch_api_data(reactions_url)

    # Flatten the reactions data if multiple reactions exist
    for reaction in reactions_data:
        reaction_id = reaction.get('id')  # Reaction ID
        name = reaction.get('name')  # Reaction user name
        type_ = reaction.get('type')  # Reaction type (like, love, etc.)

        # Append the reaction data to fb_reactions_data list
        fb_reactions_data.append({
             'post_id': post_id,  # Save the post_id
            'comment_id': comment_id,  # Save the comment_id
            'reaction_id': reaction_id,  # Reaction ID
            'name': name,  # Name of the user who reacted
            'type': type_  # Type of reaction
        })

# Convert the reactions data to a DataFrame
fb_reactions = pd.DataFrame(fb_reactions_data)


# Loop through accounts and fetch page cta for each
fb_cta_data = []

for index, row in fb_accounts.iterrows():
    page_id = row['id']  # Use the page_id from fb_accounts
    page_token = row['access_token']  # Use the page access token
    
    # Define the correct query for fetching CTA fields
    f_cta = "{page_id}/call_to_actions?fields=id,type,status,web_url,mobile_url,created_time,updated_time&access_token={PAGE_TOKEN}".format(page_id=page_id, PAGE_TOKEN=page_token)
    
    # Build the URL for fetching CTA data
    cta_url = f"{BASE_URL}{f_cta}"

    # Fetch the data
    cta_data = fetch_api_data(cta_url)

    # Unpack the CTA data into individual columns
    for cta in cta_data:
        fb_cta_data.append({
            'page_id': page_id,            # Save the page_id
            'cta_id': cta.get('id'),       # Save the CTA id
            'type': cta.get('type'),   # Save the CTA type
            'status': cta.get('status'),  # Save the CTA status
            'web_url': cta.get('web_url'), # Save the CTA web_url
            'mobile_url': cta.get('mobile_url'), # Save the mobile_url
            'created_time': cta.get('created_time'), # Save the created time
            'updated_time': cta.get('updated_time')  # Save the updated time
        })

# Convert the CTA data to a DataFrame with separate columns for each field
fb_cta = pd.DataFrame(fb_cta_data)


# Database connection settings
DB_SETTINGS = {
    'dbname': 'postgres',
    'user': 'XXX',
    'password': 'YYYY!',
    'host': 'ZZZZ',
    'port': '5432'
}

def upsert_data_into_table(dataframe, table_name, column_mapping, conflict_columns, batch_size=500):
    dataframe = dataframe.replace({np.nan: None})
    data_tuples = []

    # Prepare data tuples for insertion, including handling of JSON fields and timestamp
    for row in dataframe.to_dict(orient='records'):
        formatted_row = []
        for df_col, db_col in column_mapping.items():
            value = row.get(df_col)
            if isinstance(value, dict):  # Handle JSONB fields
                formatted_row.append(json.dumps(value))  # Serialize JSON fields
            else:
                formatted_row.append(value)  # Non-JSON fields
        # Add the current datetime for updated_at
        formatted_row.append(datetime.now())
        data_tuples.append(tuple(formatted_row))

    # Prepare the conflict columns for ON CONFLICT clause
    conflict_column_str = ', '.join(conflict_columns)

    # Exclude updated_at from being overwritten, but handle other columns
    update_columns = [f"{db_col} = EXCLUDED.{db_col}" for db_col in column_mapping.values() if db_col != 'updated_at' and db_col not in conflict_columns]

    # SQL query for upsert
    insert_query = f"""
    INSERT INTO {table_name} ({', '.join(column_mapping.values())}, updated_at) 
    VALUES %s
    ON CONFLICT ({conflict_column_str}) 
    DO UPDATE SET {', '.join(update_columns)}, updated_at = NOW()
    """

    # Execute the query in batches
    try:
        with psycopg2.connect(**DB_SETTINGS) as conn:
            with conn.cursor() as cur:
                for batch_start in range(0, len(data_tuples), batch_size):
                    batch = data_tuples[batch_start:batch_start + batch_size]
                    psycopg2.extras.execute_values(cur, insert_query, batch)
            conn.commit()
    except Exception as e:
        print(f"Error during upsert: {e}")
        raise


accounts_mapping = {
        'id': 'account_id',
        'username': 'username',
        'name': 'name',
        'profile_picture_url': 'profile_picture_url',
        'email': 'email',
        'biography': 'biography',
        'description': 'description',
        'followers_count': 'followers_count',
        'follows_count': 'follows_count',
        'website': 'website',
        'media_count': 'media_count'
}

comments_mapping = {
        'comment_id': 'comment_id',
        'reply_id': 'reply_id',
        'from_user': 'from_user',
        'message': 'message',
        'created_time': 'created_time',
        'like_count': 'like_count',
        'attachment': 'attachment',
        'parent': 'parent',
        'hidden': 'hidden'
}

page_cta_mapping = {
        'page_id': 'account_id',
        'cta_id': 'cta_id',
        'type': 'type',
        'status': 'status',
        'web_url': 'web_url',
        'mobile_url': 'mobile_url',
        'created_time': 'created_time',
        'updated_time': 'updated_time'
}

page_insights_mapping = {
        'page_id': 'account_id',
        'end_date': 'end_date',
        'page_impressions': 'page_impressions',
        'page_impressions_unique': 'page_impressions_unique',
        'page_fan_adds': 'page_fan_adds',
        'page_fan_removes': 'page_fan_removes',
        'page_fans_online': 'page_fans_online',
        'page_fans_country': 'page_fans_country',
        'page_fans_city': 'page_fans_city',
        'page_views_total': 'page_views_total',
        'page_post_engagements': 'page_post_engagements',
        'page_total_actions': 'page_total_actions',
        'page_consumptions_by_consumption_type': 'page_consumptions_by_consumption_type',
        'page_negative_feedback': 'page_negative_feedback',
        'page_negative_feedback_unique': 'page_negative_feedback_unique',
        'page_posts_impressions': 'page_posts_impressions',
        'page_posts_impressions_unique': 'page_posts_impressions_unique',
        'page_posts_impressions_paid': 'page_posts_impressions_paid',
        'page_posts_impressions_organic': 'page_posts_impressions_organic',
        'period': 'period'
}

post_comments_mapping = {
        'post_id': 'post_id',
        'comment_id': 'comment_id',
        'from_user': 'from_user',
        'message': 'message',
        'created_time': 'created_time',
        'like_count': 'like_count',
        'comment_count': 'comment_count',
        'attachment': 'attachment',
        'parent': 'parent',
        'hidden': 'hidden'
}

post_insights_mapping = {
        'post_id': 'post_id',
        'account_id': 'account_id',
        'end_date': 'end_date',
        'post_impressions': 'post_impressions',
        'post_clicks': 'post_clicks',
        'post_impressions_paid': 'post_impressions_paid',
        'post_activity_by_action_type': 'post_activity_by_action_type',
        'post_clicks_by_type': 'post_clicks_by_type',
        'page_consumptions_by_consumption_type': 'page_consumptions_by_consumption_type',
        'period': 'period'
}

posts_mapping = {
        'post_id': 'post_id',
        'page_id': 'account_id',
        'promotion_status': 'promotion_status',
        'scheduled_publish_time': 'scheduled_publish_time',
        'message': 'message',
        'message_tags': 'message_tags',
        'created_time': 'created_time',
        'permalink_url': 'permalink_url',
        'status_type': 'status_type',
        'call_to_action': 'call_to_action',
        'instagram_eligibility': 'instagram_eligibility',
        'is_published': 'is_published',
        'is_hidden': 'is_hidden',
        'ad_status': 'ad_status',
        'ads_run': 'ads_run'
}

reactions_mapping = {
    'comment_id': 'comment_id',
    'reaction_id': 'reaction_id',
    'name': 'name',
    'type': 'type'
}


#fb_accounts, 'fb.accounts', accounts_mapping, conflict_columns=['account_id']
#fb_page_insights, 'fb.page_insights', page_insights_mapping, conflict_columns=['account_id','end_date']
#fb_posts, 'fb.posts', posts_mapping, conflict_columns=['post_id']
#fb_post_insight, 'fb.post_insights', post_insights_mapping, conflict_columns=['post_id','end_date']
#fb_post_comments, 'fb.post_comments', comments_mapping, conflict_columns=['post_id', 'comment_id','created_time']
#fb_comments, 'fb.comments', comments_mapping, conflict_columns = ['comment_id','created_time']
#fb_reactions, 'fb_reactions', reactions_mapping, conflict_columns = ['reaction_id']
#fb_cta, 'fb.page_cta', page_cta_mapping, conflict_columns - ['cta_id']

upsert_data_into_table(fb_accounts, 'fb.accounts', accounts_mapping, conflict_columns=['account_id'])
upsert_data_into_table(fb_page_insights, 'fb.page_insights', page_insights_mapping, conflict_columns=['account_id','end_date'])
upsert_data_into_table(fb_posts, 'fb.posts', posts_mapping, conflict_columns=['post_id'])
upsert_data_into_table(fb_post_insights, 'fb.post_insights', post_insights_mapping, conflict_columns=['post_id','end_date'])
upsert_data_into_table(fb_post_comments, 'fb.post_comments', post_comments_mapping, conflict_columns=['post_id', 'comment_id','created_time'])
upsert_data_into_table(fb_comments, 'fb.comments', comments_mapping, conflict_columns = ['comment_id', 'reply_id','created_time'])
upsert_data_into_table(fb_reactions, 'fb.reactions', reactions_mapping, conflict_columns = ['reaction_id'])
upsert_data_into_table(fb_cta, 'fb.page_cta', page_cta_mapping, conflict_columns = ['cta_id'])


Error fetching data: 403 Client Error: Forbidden for url: https://api.linkedin.com/v2/shares?q=owners&owners=urn:li:organization:YOUR_ORGANIZATION_ID&sortBy=LAST_MODIFIED&sharesPerOwner=100
No posts data available for organization YOUR_ORGANIZATION_ID.
No posts found for this organization.
